In [17]:
import numpy as np;
import matplotlib.pyplot as plt;
import sympy;
import scipy;
from sympy.geometry import Point, Ray, Circle, intersection;

# Normal distribution

In [18]:
np.random.seed(0);

N = 10
cutoff = 4.0         # To make sure it doest go passed lens
mu= 0
sigma = 2            # Change to w(1) = w(0)^2 (1 + (lambda/pi w_0)^2)
X = scipy.stats.truncnorm(
    (-cutoff) / sigma, (cutoff) / sigma, loc=mu, scale=sigma)

sample= X.rvs(size=N)
ray_directions = [Point(1, sample[i]) for i in range(N)]
print(ray_directions)

AttributeError: module 'scipy' has no attribute 'stats'

# Ray grid propagation

In [5]:
from sympy.geometry import Polygon;
box = Polygon(Point(10,10),Point(30,10),Point(30,-10),Point(10,-10));

In [6]:
def ray_grid_propagation(enter, exit, N, c):
    """
    Calculates contribution of a ray entering at enter and leaving at exit,
    on an N x N grid.
    enter and exit must be normalized to [0,1] x [0,1].
    """
    steps = c*N;
    A_sub = np.zeros((N,N));
    for i in range(0, steps):
        per = i/steps;
        current = enter+per*(exit-enter);
        x_ind = min(int(current[0]*N),N-1);
        y_ind = min(int(current[1]*N),N-1);
        A_sub[x_ind, y_ind] = A_sub[x_ind, y_ind]+1;

    return A_sub/(c);

In [ ]:
N = 101;
A = np.zeros((N,N));
box = Polygon(Point(10,10),Point(30,10),Point(30,-10),Point(10,-10));

ray_count = 100;
for i in range(0,ray_count):
    y = np.random.normal(scale = 5.0);
    current_ray_origin = Point(0,y);
    current_ray_direction = Point(2,-0.1*y);
    current_ray = Ray(current_ray_origin, current_ray_origin + current_ray_direction);

    box_intersection = intersection(box,current_ray);
    if(len(box_intersection) == 2):
        begin = np.array([float(box_intersection[0].x),float(box_intersection[0].y)]);
        end = np.array([float(box_intersection[1].x),float(box_intersection[1].y)]);
        begin_transformed = (begin - [10,-10])/20.0;
        end_transformed = (end - [10,-10])/20.0;
        A_sub = ray_grid_propagation(begin_transformed, end_transformed, N, 2);
        A = A + A_sub;

A = A/ray_count;
plt.matshow(A);
plt.colorbar();

In [ ]:
import ipywidgets as widgets;
from IPython.display import display;

w = widgets.IntSlider(min = 0, max = N-1, step=1, description="slice_z");

def proc(slice_z):
    plt.plot(A[slice_z,:]);
    
out = widgets.interactive_output(proc, {'slice_z': w});

display(w);
display(out);